In [227]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from category_encoders import OneHotEncoder, TargetEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [228]:
DATA_PATH = './data/'

In [287]:
###DATA WRANGLING/EDA

def wrangle(data):
    df = pd.read_csv(data,
                parse_dates=['last_scraped','host_since']
               ).set_index('last_scraped')
    #Dropping unecessary high cardinality columns
    df.drop(columns=['id','listing_url','scrape_id','name',
                    'description','neighborhood_overview','picture_url',
                    'host_id','host_url','host_location',
                    'host_about','host_response_rate','host_thumbnail_url',
                    'host_picture_url','host_name','host_about',
                    'host_acceptance_rate','host_listings_count',
                    'host_neighbourhood','host_total_listings_count',
                    'host_verifications','neighbourhood','neighbourhood_cleansed',
                    'neighbourhood_group_cleansed','property_type','bathrooms_text',
                    'bathrooms', 'amenities','calendar_updated','calendar_last_scraped', 
                    'first_review', 'last_review','license','calculated_host_listings_count',
                    'calculated_host_listings_count_entire_homes',
                    'calculated_host_listings_count_private_rooms',
                    'calculated_host_listings_count_shared_rooms',
                    'host_has_profile_pic'], inplace=True)
    
    #Converted t and f object type into 1 and 0
    df.replace('t',1, inplace=True)
    df.replace('f',0, inplace=True)
    df['host_response_time'].fillna('within an hour',inplace=True)
    df['Host_experience_in_years'] = 2020 - df['host_since'].dt.year
    df.drop(columns=['host_since'], inplace=True)
    df['price']  = df['price'].str.strip('$')
    df['price']  = df['price'].str.replace(',','').astype(float)
    
    #Dropping outliers
    #Change high value into NaN
    df['price'] = df.query("price < 20000")
    
    
    
    

    
    return df

In [288]:
df = wrangle(DATA_PATH+'listings.csv')



ValueError: cannot reindex from a duplicate axis

In [ ]:
df.describe(exclude='number')

In [ ]:
df['host_response_time'].value_counts()

In [ ]:
df['price'].dtype

In [ ]:
df.info()

In [ ]:
###SPLITTING DATA
target = 'price'
y = df[target]
X = df.drop(columns=target)

In [ ]:
cutoff = '2021-01-04'
mask = X.index < cutoff
X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]

In [ ]:
y_test.describe()

In [237]:
###ESTABLISHING BASELINE
print('Mean AirBnB Price:', y_train.mean())
y_pred = [y_train.mean()] * len(y_train)
print('Baseline MAE:', mean_absolute_error(y_train, y_pred))
#add rmse
#add r2 score

Mean AirBnB Price: 13920.955987268137
Baseline MAE: 10814.345358018792


In [277]:
###BUILDING MODEL


model_rfr = make_pipeline(
    OneHotEncoder(), 
    SimpleImputer(strategy='median'), 
    RandomForestRegressor(n_estimators=25, random_state=42)
)


model_rfr.fit(X_train, y_train);



c:\users\marvin\.virtualenvs\unit-2-proj-mmzadrit\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [280]:
y_pred_train = model_rfr.predict(X_train)
y_pred_test = model_rfr.predict(X_test)
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)


print(train_mae, test_mae)

3036.4028536933374 10055.207755235544


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11353 entries, 2021-01-04 to 2021-01-01
Data columns (total 37 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   host_since                   11337 non-null  datetime64[ns]
 1   host_response_time           9968 non-null   object        
 2   host_is_superhost            11337 non-null  float64       
 3   host_identity_verified       11337 non-null  float64       
 4   latitude                     11353 non-null  float64       
 5   longitude                    11353 non-null  float64       
 6   room_type                    11353 non-null  object        
 7   accommodates                 11353 non-null  int64         
 8   bedrooms                     10332 non-null  float64       
 9   beds                         11286 non-null  float64       
 10  price                        11353 non-null  object        
 11  minimum_nights          